In [77]:
#Imports
import pandas as pd
from itertools import product

In [78]:
# Constants for points
WIN_POINTS = 3
TIE_POINTS = 1
LOSS_POINTS = 0

# Division 1 current score
current_scores = {
    'Grizzled Gamblers': 8,  # Replace these with actual starting scores
    'Zephyr Loch Ness Monsters': 8,
    'Duivelsei Academy': 5,
    'Dorans Lars de Vries': 4,
    'Green Tea Love': 1,
    'Zephyr Kratos': 1,
    'Esports Team Twente': 1
}

# # Division 2 current score
# current_scores = {
#     'Iron Bucket Boys': 5,  # Replace these with actual starting scores
#     'Blitzfrank': 5,
#     'Zephyr Shiva': 5,
#     'Peritan the Platypus': 4,
#     'Zephyr Hydra': 4,
#     'TSEA Link Reborn': 0
# }

# # Division 3 current score
# current_scores = {
#     'Zephyr Odysseus': 9,  # Replace these with actual starting scores
#     'BS Scaling': 5,
#     'U-Padvinders': 3,
#     'Dorans Dirty Little Animals': 2,
#     'BS Happy Hour': 2,
#     'DEI 4Fun Boomers': 1
# }

# Division 4 current score
# current_scores = {
#     'Ignite Green': 9,  # Replace these with actual starting scores
#     'Zephyr Ouroboros': 6,
#     'Zephyr Kerberos': 4,
#     'Infinite Spirits': 3,
#     'Boosted Chickens': 3,
#     'TSEA Pink Wards': 1
# }

# # Division 5 current score
# current_scores = {
#     'Dorans WDATCKG': 9,  # Replace these with actual starting scores
#     'Boosted Kittens': 6,
#     'Ignite Purple': 4,
#     'TSEA Link Happy Dumplings': 3,
#     'Infinite Pixels': 3,
#     'Ignite Diamond': 1
# }

file_location = 'Division 1.xlsx'

In [79]:
def read_matches(file_path):
    df = pd.read_excel(file_path)
    return [(row['Team1'], row['Team2']) for index, row in df.iterrows()]

In [80]:
def simulate_matches(matches, current_scores):
    outcomes = list(product([WIN_POINTS, TIE_POINTS, LOSS_POINTS], repeat=len(matches)))
    standings = []
    for outcome in outcomes:
        scores = current_scores.copy()
        for i, match in enumerate(matches):
            team1, team2 = match
            result = outcome[i]
            if result == WIN_POINTS:
                scores[team1] += WIN_POINTS
            elif result == TIE_POINTS:
                scores[team1] += TIE_POINTS
                scores[team2] += TIE_POINTS
            elif result == LOSS_POINTS:
                scores[team2] += WIN_POINTS
        standings.append(scores)
    return standings

In [81]:
def determine_top_teams(standings, num_teams):
    position_totals = {team: 0 for team in standings[0]}
    top4_counts = {team: 0 for team in standings[0]}
    total_scenarios = len(standings)

    for scenario in standings:
        sorted_scenario = sorted(scenario.items(), key=lambda item: item[1], reverse=True)
        for position, (team, _) in enumerate(sorted_scenario, start=1):
            position_totals[team] += position
            if position <= num_teams:
                top4_counts[team] += 1

    average_positions = {team: total / total_scenarios for team, total in position_totals.items()}
    top4_percentages = {team: (count / total_scenarios) * 100 for team, count in top4_counts.items()}

    return average_positions, top4_percentages

In [82]:
def calculate_final_position_probabilities(standings):
    # Initialize a dictionary to hold the count of final positions for each team
    position_counts = {team: {i: 0 for i in range(1, len(standings[0]) + 1)} for team in standings[0]}
    
    # Count the occurrences of each final position for each team
    for scenario in standings:
        sorted_scenario = sorted(scenario.items(), key=lambda item: item[1], reverse=True)
        for position, (team, _) in enumerate(sorted_scenario, start=1):
            position_counts[team][position] += 1

    # Calculate the probability distribution based on the counts
    total_scenarios = len(standings)
    position_probabilities = {team: {position: count / total_scenarios * 100 for position, count in positions.items()} for team, positions in position_counts.items()}

    return position_probabilities


In [83]:
def calculate_clutch_factor(file_path, top_x, current_scores):
    original_matches = read_matches(file_path)
    baseline_standings = simulate_matches(original_matches, current_scores)
    _, baseline_top_x_chances = determine_top_teams(baseline_standings, top_x)

    clutch_factors = []

    for match in original_matches:
        modified_matches = original_matches.copy()
        modified_matches.remove(match)

        # Simulate the scenario where the first team wins
        scores_first_team_wins = current_scores.copy()
        scores_first_team_wins[match[0]] += WIN_POINTS
        standings_first_team_wins = simulate_matches(modified_matches, scores_first_team_wins)
        _, top_x_chances_first_team_wins = determine_top_teams(standings_first_team_wins, top_x)

        # Simulate the scenario where the second team wins
        scores_second_team_wins = current_scores.copy()
        scores_second_team_wins[match[1]] += WIN_POINTS
        standings_second_team_wins = simulate_matches(modified_matches, scores_second_team_wins)
        _, top_x_chances_second_team_wins = determine_top_teams(standings_second_team_wins, top_x)

        # Calculate clutch factor
        clutch_factor_first_team = abs(baseline_top_x_chances[match[0]] - top_x_chances_first_team_wins[match[0]])
        clutch_factor_second_team = abs(baseline_top_x_chances[match[1]] - top_x_chances_second_team_wins[match[1]])

        clutch_factors.append((match, {
            match[0]: clutch_factor_first_team,
            match[1]: clutch_factor_second_team
        }))

    # Sort matches by the sum of clutch factors for both teams
    clutch_factors.sort(key=lambda x: sum(x[1].values()), reverse=True)
    
    return clutch_factors

In [84]:
def main(file_path, top_x, current_scores):
    with open('output.txt', 'w') as file:  # Open file for writing, overwrites the file if exists
        def print_both(*args, **kwargs):
            print(*args, **kwargs)  # Print to console
            print(*args, **kwargs, file=file)  # Write to file
        
        matches = read_matches(file_path)
        all_standings = simulate_matches(matches, current_scores)
        average_positions, top4_percentages = determine_top_teams(all_standings, top_x)
        position_probabilities = calculate_final_position_probabilities(all_standings)

        # Output average positions and top 4 percentages
        print_both("Average Finishing Positions:")
        for team, avg_position in average_positions.items():
            print_both(f"{team}: {round(avg_position,2)}")
        print_both("-" * 20)

        print_both("Percentage of Times Teams Make the Top", top_x)
        for team, percentage in top4_percentages.items():
            print_both(f"{team}: {round(percentage,2)}%")
        print_both("-" * 20)

        # New output for probability distribution of final positions
        print("Probability Distribution of Final Positions:")
        for team, probabilities in position_probabilities.items():
            print(f"{team}:")
            for position, probability in sorted(probabilities.items()):
                print(f"  Position {position}: {round(probability, 2)}%")
            print("-" * 20)

        # Calculate and output the clutch factor for each match
        print_both("Clutch Factor Analysis:")
        clutch_factors = calculate_clutch_factor(file_path, top_x, current_scores)
        for match, factors in clutch_factors:
            print_both(f"Match: {match[0]} vs {match[1]}")
            for team, factor in factors.items():
                direction = "positive" if factor > 0 else "negative"
                print_both(f"{team}: Clutch Factor: {direction} ({round(factor, 2)}%)")
            print_both()  # New line for readability

main(file_location, 4, current_scores)


Average Finishing Positions:
Grizzled Gamblers: 1.86
Zephyr Loch Ness Monsters: 2.05
Duivelsei Academy: 3.29
Dorans Lars de Vries: 3.97
Green Tea Love: 4.98
Zephyr Kratos: 5.86
Esports Team Twente: 5.98
--------------------
Percentage of Times Teams Make the Top 4
Grizzled Gamblers: 99.44%
Zephyr Loch Ness Monsters: 99.33%
Duivelsei Academy: 80.39%
Dorans Lars de Vries: 65.31%
Green Tea Love: 30.53%
Zephyr Kratos: 12.99%
Esports Team Twente: 12.01%
--------------------
Probability Distribution of Final Positions:
Grizzled Gamblers:
  Position 1: 48.02%
  Position 2: 25.15%
  Position 3: 19.8%
  Position 4: 6.47%
  Position 5: 0.56%
  Position 6: 0.0%
  Position 7: 0.0%
--------------------
Zephyr Loch Ness Monsters:
  Position 1: 35.85%
  Position 2: 32.33%
  Position 3: 23.53%
  Position 4: 7.62%
  Position 5: 0.67%
  Position 6: 0.0%
  Position 7: 0.0%
--------------------
Duivelsei Academy:
  Position 1: 8.83%
  Position 2: 22.1%
  Position 3: 26.89%
  Position 4: 22.58%
  Position 